# qe_test.ipynb
### Kat Nykiel
## Objectives
- query structure from materials project
- strain structure using pymatgen
- generate QE input files with pymatgen
- submit QE job

### Query structure from materials project (new API)

In [ ]:
from mp_api.client import MPRester

In [ ]:
# Read in new Materials Project API key
with open(os.path.expanduser('~/keys.txt'), 'r') as f:
    key = f.readlines()[0]
    
# Query using new API for material with ID of mp-149 (Si)
with MPRester(key) as m:
    data = m.summary.search(material_ids=["mp-149"])

In [ ]:
struct = data[0].structure
print(struct)

### Strain structure using pymatgen

In [ ]:
# Check lattice parameters before strain
struct.lattice.abc

In [ ]:
# Apply strain to lattice
struct.apply_strain(0.1)

In [ ]:
# Check new lattice vectors
struct.lattice.abc

### Generate QE input files using pymatgen

In [ ]:
from pymatgen.io.pwscf import PWInput
input_set = PWInput(struct, pseudo={'Si':'Si.PBE.UPF'})
input_set.write_file(filename='Si.in')

In [ ]:
! < Si.in > Si.out

### Extract QE outputs using pymatgen

In [ ]:
from pymatgen.io.pwscf import PWOutput
output_set = PWOutput('Si.out')

In [ ]:
output_set.final_energy

In [ ]:
[print(k) for k,v in output_set.data.items()]

In [ ]:
# Extract outputs using monty's regrep method
output_dict = output_set.read_pattern(patterns={'energy':r"energy\(sigma->0\)\s+=\s+([\d\-.]+)"})